# Prompt templates and direct call api

First part of the chain, just setup api key (on `azure` in this example). This cell prints the name of the base deployment just to verify that we correctly loaded the .env file.

Remember you need a .env file with the following content:

```
OPENAI_API_KEY=xxxxxxxxx
OPENAI_API_BASE=https://alkopenai2.openai.azure.com/
HUGGINGFACEHUB_API_TOKEN=xxxxxxx
PINECONE_KEY=xxxxxxx
PINECONE_ENV=us-west1-gcp-free
SERPAPI_API_KEY=xxxxx
```

In [11]:
import os
import openai
from pprint import pprint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"

# Remember that you need to set the OPENAI_API_BASE to point openai to your specific deployment.

openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# print base to verify you are pointint to the right deployment
print(openai.api_base)


https://alkopenai2.openai.azure.com/


Now to work langchain nees to work with LLM, you start creating some LLM in this situation we use OpenAI in azure in different flavors.

The important aspect is that some of the llm models are `chat models` and others are `text models`. The chat models are trained to answer questions and the text models are trained to generate text. You will use both of these in **different situation**.

In [12]:
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI  

# Pay attention that gpt35 and gpt4 are chat based llms
gpt35 =  AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    deployment_name="Gpt35", 
    model_name="gpt-35-turbo"
)

gpt4 =  AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    deployment_name="Gpt4", 
    model_name="gpt-4"
)

# this is not a Chat model this is a text model
davinci = AzureOpenAI(
	temperature=0,
	openai_api_version="2023-03-15-preview",
    deployment_name="text-davinci-003", 
	model_name="text-davinci-003"
)

Langchain first function: ability to create a `prompt` with placeholders to better manage the prompt. The key here is simply the ability to create a string with placeholders.

In [13]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate

# pay attention we are not using python f" style of strings.
prompt_template_string = """Translate sentence surrounded by triple ticks to {target_lang}.

```{sentence}```
"""

# Create a prompt object from the template 
# super important THE MODEL IS OF TYPE CHATPROMPTTEMPLATE
chat_prompt_template = ChatPromptTemplate.from_template(prompt_template_string)

# now you have a complex template object that as an example has input variables
pprint(chat_prompt_template.messages[0].input_variables)

# now you can generate a real prompt substituing input variables.
chat_sample_prompt = chat_prompt_template.format_messages(target_lang="de", sentence="This is a simple experiment to test langchain")
pprint(f"Type of the chat_sample_prompt is a list of {len(chat_sample_prompt)} first element type is  {type(chat_sample_prompt[0])}")

# you can create a simple template
prompt_template = PromptTemplate.from_template(prompt_template_string)

pprint(f"Type of the prompt_template is {type(prompt_template)} and input variables are {prompt_template.input_variables}")
sample_prompt = prompt_template.format(target_lang="de", sentence="This is a simple experiment to test langchain")

pprint(f"Type of the sample_prompt is {type(sample_prompt)}")
pprint(sample_prompt   )

['sentence', 'target_lang']
('Type of the chat_sample_prompt is a list of 1 first element type is  <class '
 "'langchain.schema.HumanMessage'>")
('Type of the prompt_template is <class '
 "'langchain.prompts.prompt.PromptTemplate'> and input variables are "
 "['sentence', 'target_lang']")
"Type of the sample_prompt is <class 'str'>"
('Translate sentence surrounded by triple ticks to de.\n'
 '\n'
 '```This is a simple experiment to test langchain```\n')


In [14]:
# this will show that the chat sample prompt is a list of objects and each object has a type
# and a content attribute that contains the text of the prompt
first_element = chat_sample_prompt[0]
pprint(f"Type of first element: {type(first_element)}")
print(first_element.content)

"Type of first element: <class 'langchain.schema.HumanMessage'>"
Translate sentence surrounded by triple ticks to de.

```This is a simple experiment to test langchain```



`Chat Models` can accept the textPrompt object, that is basically an array, and simply return another piece of the conversation. We are actually creating a `prompt` with a single message and the chat model will answer with another piece of object.

You will see that the textPrompt is a `list` containing HumanMessages or AIMessages. This is the standard way to dialogate with a chat model.

In [15]:
# now you can call an api through the wrapper
print(f"textPrompt is of type {type(chat_sample_prompt)} and first element is of type {type(chat_sample_prompt[0])}")  
result = gpt35(chat_sample_prompt)  
print(f"Result is of type {type(result)}")
pprint(result)  

#actually it is more useful to print the content
pprint(f"Content of the answer is = {result.content}")

textPrompt is of type <class 'list'> and first element is of type <class 'langchain.schema.HumanMessage'>
Result is of type <class 'langchain.schema.AIMessage'>
AIMessage(content='```Dies ist ein einfaches Experiment, um Langkette zu testen```', additional_kwargs={}, example=False)
('Content of the answer is = ```Dies ist ein einfaches Experiment, um '
 'Langkette zu testen```')


Davinci is a simple completion model, so it will simply complete my prompt. 
1. I need to pass the prompt as string not the entire prompt (only chain models supports    the entire prompt)
2. I will simply return text

In [16]:
# if you have a ChatPromptTemplate you need to pass a single string content.
result = davinci(chat_sample_prompt[0].content)
print(result)

# it is more standard to have a standard PromptTemplate that uses .format to create
# a sample string
result = davinci(sample_prompt)
pprint(result)


Dies ist ein einfaches Experiment, um Langchain zu testen.
'\nDies ist ein einfaches Experiment, um Langchain zu testen.'


Basically `ChatPromptTemplate` and `PromptTemplate` are used respectively with models of type Chat or Text. Please always remember this distinction between the two.

# Starting using a chat model

Now we will start using a chat model to simulate what is happening in standard chat GPT. Situation becomes more interesting because we can now interact with `AIMessage, HumanMessage and SystemMessage` objects.

In [17]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# This example is just wrong, each call is a different conversation, a different api
# call. 
result1 = gpt35([HumanMessage(content =      "Hy my name is gian maria")])
print(result1)
result2 = gpt35([HumanMessage(content =      "What is my name")])
print(result2)

content='Hello Gian Maria, how can I assist you today?' additional_kwargs={} example=False
content="I'm sorry, but as an AI language model, I do not have access to your personal information such as your name." additional_kwargs={} example=False


Call to API have no context, each call is a new context, a new call and the AI does not know nothing about previous calls. It is up to the caller to provide `a list of the previous calls`.

In [18]:
conversation = []
conversation.append(SystemMessage(content="You are an helpful UI assistant"))
conversation.append(HumanMessage(content="Hello My name is Gian Maria"))
ai_answer = gpt35(conversation)
pprint(f"AI answer with an object of type {type(ai_answer)}")
pprint(ai_answer.content)
#Now we append the AI answer to the conversation
conversation.append(ai_answer)

# now we can continue the conversation asking for my name
message = HumanMessage(content="What is my name?")
pprint(message.content)
conversation.append(message)
ai_answer = gpt35(conversation)
pprint(ai_answer.content)

"AI answer with an object of type <class 'langchain.schema.AIMessage'>"
'Hello Gian Maria! How may I assist you today?'
'What is my name?'
('Your name is Gian Maria, as you mentioned earlier. Is there anything else I '
 'can help you with?')


You can now create a method that will simply print the whole chat.

In [19]:
def print_chat(conversation):
    for i in range(len(conversation)):
        # each element is of three different type, human, context and ai we need
        # to distinguish them based on type of object
        if isinstance(conversation[i], SystemMessage):
            print("Context: " + conversation[i].content)
        elif isinstance(conversation[i], HumanMessage):
            print("Human: " + conversation[i].content)
        else:
            print("AI: " + conversation[i].content)

def start_chat(model, context, first_human_message):
    conversation = [SystemMessage(content=context)]
    conversation.append(HumanMessage(content=first_human_message))
    conversation.append(model(conversation))
    return conversation

def continue_chat(model, conversation, human_message):
    conversation.append(HumanMessage(content=human_message))
    conversation.append(model(conversation))
    return conversation



Previous methods allows you to interact easier with the chat

In [20]:
context = start_chat(gpt35, "you are an helpful ai", "Hi my name is Gian Maria")
print_chat(context)

pprint("------------------")
# now you can prosecute the chat
context = continue_chat(gpt35, context, "What is my name?")
print_chat(context)

Context: you are an helpful ai
Human: Hi my name is Gian Maria
AI: Hello Gian Maria! How can I assist you today?
'------------------'
Context: you are an helpful ai
Human: Hi my name is Gian Maria
AI: Hello Gian Maria! How can I assist you today?
Human: What is my name?
AI: Your name is Gian Maria.


Since you are in full control of everythign you can manipulate everything.

In [21]:
context = start_chat(gpt35, "you are an helpful ai", "Hi my name is Gian Maria. What is your name?")
print_chat(context)

pprint("------------------")
# now you can prosecute the chat but you can change anything like the previous answer of the ai
context[2].content = "Hi Gian Maria, my name is Ambrogio."
context = continue_chat(gpt35, context, "Which are the letters that are common between our names? Please explain citing yours and my name.")
print_chat(context)

Context: you are an helpful ai
Human: Hi my name is Gian Maria. What is your name?
AI: Hello Gian Maria! I am an AI language model developed by OpenAI, and I don't have a name. You can call me OpenAI if you'd like! How can I assist you today?
'------------------'
Context: you are an helpful ai
Human: Hi my name is Gian Maria. What is your name?
AI: Hi Gian Maria, my name is Ambrogio.
Human: Which are the letters that are common between our names? Please explain citing yours and my name.
AI: The letters that are common between our names, Gian Maria and Ambrogio are "a", "i" and "m". Gian Maria has the letters "a", "i" and "m" in it, and Ambrogio has the letters "a", "i" and "m" in it as well.


Finally, you can use a ChatPromptTemplate for a complex conversation

In [23]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful AI bot. Your name is {ai_name}."),
    HumanMessagePromptTemplate.from_template("Hi my name is {user_name}."),
    AIMessagePromptTemplate.from_template("Hi {user_name}, nice to meet you, my name is {ai_name} and I'm here to help you!"),
    HumanMessagePromptTemplate.from_template("{user_input}"),
])

prompt = template.format_messages(
    ai_name="Al2000",
    user_name="Gian Maria",
    user_input="I'm having some problems with my computer, can you help me? But first, what's your name?"
)

answer = gpt35(prompt)
print(answer)

content='As I mentioned earlier, my name is Al2000. And yes, I would be glad to help you with any computer problems you may be experiencing. What seems to be the issue?' additional_kwargs={} example=False


You can change some other paramter of the query directly in the call

In [33]:
prompt = template.format_messages(
    ai_name="Al2000",
    user_name="Gian Maria",
    user_input="""I have a computer that suddenly does not boot to window. 
    Monitor turns on, windows logo appears but then nothing happens. 
    What can I do to diagnose the problem?"""
)

print(prompt)

answer = gpt35(prompt, temperature=2)
print(answer)

[SystemMessage(content='You are a helpful AI bot. Your name is Al2000.', additional_kwargs={}), HumanMessage(content='Hi my name is Gian Maria.', additional_kwargs={}, example=False), AIMessage(content="Hi Gian Maria, nice to meet you, my name is Al2000 and I'm here to help you!", additional_kwargs={}, example=False), HumanMessage(content='I have a computer that suddenly does not boot to window. \n    Monitor turns on, windows logo appears but then nothing happens. \n    What can I do to diagnose the problem?', additional_kwargs={}, example=False)]
